In [1]:
import keras
import os
from keras.models import Sequential
from keras.layers import Dense, Dropout, Dense, Input, concatenate,Conv2D, MaxPool2D
from keras import backend as K

import numpy as np
import tensorflow as tf
import random
import cv2
import os
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.layers import Input,Flatten,Dropout,Conv2D,MaxPooling2D,GlobalAveragePooling2D
from keras.layers import Activation
from tensorflow.keras.optimizers import Adam,RMSprop,SGD
from keras.metrics import categorical_crossentropy
from keras.layers import BatchNormalization
from sklearn.metrics import confusion_matrix
from keras.models import Model
from keras import callbacks
from keras.callbacks import TensorBoard
from keras import regularizers
import pickle
from sklearn.preprocessing import LabelBinarizer
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import time
import matplotlib.pyplot as plt
import keras
import datetime


In [7]:


def inception_module(x,NF_Left, NF_Middle_1, NF_Middle_2, NF_right_2 ,name = None):
    coV_1x1 = Conv2D(NF_Left, (1, 1), padding='same', activation='relu', name = name+'_left',use_bias=False)(x)
    
    coV_3x3 = Conv2D(NF_Middle_1, (1, 1), padding='same', activation='relu', name = name+'_middle_1',use_bias=False)(x)
    coV_3x3 = Conv2D(NF_Middle_2, (3, 3), padding='same', activation='relu', name = name+'_middle_2',use_bias=False)(coV_3x3)
    
    coV_Max_3x3 = MaxPooling2D((3, 3), strides=(1, 1), padding='same', name = name+'_right_1')(x)
    coV_Max_3x3 = Conv2D(NF_right_2, (1, 1), padding='same', activation='relu', name = name+'_right_2',use_bias=False)(coV_Max_3x3)
    
    output = concatenate([coV_1x1, coV_3x3, coV_Max_3x3], axis=3, name=name)
    
    return output


In [8]:
input_layer = Input(shape = (32,32,3), name = 'input_layer')

x = Conv2D(32, (5, 5), input_shape=(32, 32, 3), activation='relu',padding="valid",strides=(1, 1), name = 'conv_1',use_bias=False)(input_layer)
x = inception_module(x,NF_Left = 32, NF_Middle_1 = 32, NF_Middle_2 = 64, NF_right_2 = 32,name = 'module_1')
x = inception_module(x,NF_Left = 64, NF_Middle_1 = 64, NF_Middle_2 = 128, NF_right_2 = 64,name = 'module_2')
x = MaxPooling2D( pool_size=(2, 2), strides=(2, 2), padding="same")(x)
x = Flatten()(x)
x = Dense(256,activation = 'relu',use_bias=False)(x)
x = Dense(10,activation='softmax',use_bias=False)(x)
model = Model(input_layer,x)
model.summary()



Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_layer (InputLayer)        [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv_1 (Conv2D)                 (None, 28, 28, 32)   2400        input_layer[0][0]                
__________________________________________________________________________________________________
module_1_middle_1 (Conv2D)      (None, 28, 28, 32)   1024        conv_1[0][0]                     
__________________________________________________________________________________________________
module_1_right_1 (MaxPooling2D) (None, 28, 28, 32)   0           conv_1[0][0]                     
______________________________________________________________________________________________

In [15]:
# output size, param # 
input_layer = Input(shape = (81,81,3), name = 'input_layer')

x = Conv2D(32, (5, 5), input_shape=(81,81,3), activation='relu',padding="same",strides=(1, 1), name = 'conv_1',use_bias=False)(input_layer)
x = Conv2D(32, (7, 7), activation='relu',padding="same",strides=(1, 1), name = 'conv_2',use_bias=False)(x)
x = MaxPooling2D(pool_size=(4, 4), strides=(3, 3), padding="valid")(x)
x = Flatten()(x)
x = Dense(10,activation = 'relu',use_bias=False)(x)
# x = Dense(10,activation='softmax',use_bias=False)(x)
model = Model(input_layer,x)
model.summary()


Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     [(None, 81, 81, 3)]       0         
_________________________________________________________________
conv_1 (Conv2D)              (None, 81, 81, 32)        2400      
_________________________________________________________________
conv_2 (Conv2D)              (None, 81, 81, 32)        50176     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 26, 26, 32)        0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 21632)             0         
_________________________________________________________________
dense_7 (Dense)              (None, 10)                216320    
Total params: 268,896
Trainable params: 268,896
Non-trainable params: 0
_____________________________________________________

In [3]:
n_mult_conv1 = 5808 * 214 * 214
print(n_mult_conv1)


265983168


In [5]:
# Depthwise Separable Convolution
def dep_sep_conv(input, n_filter, filter_size, stride):
    
    para_ds1 = 3 * 11 * 11
    mult_ds1 = 3 * 11 * 11 * 214 * 214 
    para_ds2 = 16 * 1 * 1 * 3 
    mult_ds2 = 16 * 3 * 1 * 1 * 214 * 214 






In [6]:
import math

input_size = 4.0
kernel = 3.0
stride = 2.0
# no padding
output_size = math.ceil((input_size - kernel + 1)/stride)
print('output size: ' + str(output_size))
print((input_size - kernel + 1)/stride)

output size: 1
1.0


In [7]:
#transpose conv output size
#o = ceil((i - k + 1)/s), o=2, k =3, s = 2

stride = 2
H1 = 2
HF = 3
# Padding==Same:
H = H1 * stride
print(H)
# Padding==Valid
H = (H1-1) * stride + HF
print(H)

4
5
